In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import implicit

### 데이터 준비 및 전처리

In [2]:
rating_file_path = os.getenv('HOME') + '/aiffel/recommend_system/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()



,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남김
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 변경
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommend_system/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings['movie_id'].nunique()

3628

In [8]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [9]:
# 영화 title 이 있는 데이터프레임과 병함
movie_ratings = pd.merge(ratings, movies[['title','movie_id']], on='movie_id', how='left')
movie_ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


### 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [10]:
movie_ratings['title'].nunique()

3628

In [11]:
movie_ratings['user_id'].nunique()

6039

In [12]:
popular_movies = movie_ratings.groupby('title')['counts'].count()
popular_movies.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 내가 선호하는 영화 5가지를 골라서 ratings에 추가

In [13]:
my_favorite = ['Jurassic Park (1993)', 'Forrest Gump (1994)', '2001: A Space Odyssey (1968)', 'Contact (1997)', 'Gladiator (2000)']

# user_id가 lee가 위 영화 평점을 5점 줬다고 가정
my_movies = pd.DataFrame({'user_id':['lee']*5, 'title':my_favorite, 'counts':[5]*5})

if not movie_ratings.isin({'user_id':['lee']})['user_id'].any():
    movie_ratings = movie_ratings.append(my_movies)
movie_ratings.tail(10)

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,lee,NaN,5,NaN,Jurassic Park (1993)
1,lee,NaN,5,NaN,Forrest Gump (1994)
2,lee,NaN,5,NaN,2001: A Space Odyssey (1968)
3,lee,NaN,5,NaN,Contact (1997)
4,lee,NaN,5,NaN,Gladiator (2000)


In [14]:
# 인덱싱
userid_unique = movie_ratings['user_id'].unique()
movie_unique = movie_ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [15]:
# 인덱싱 확인
print(user_to_idx['lee'])
print(movie_to_idx['Forrest Gump (1994)'])

6039
160


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movie_ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movie_ratings):
    print('movie_id column indexing OK!!')
    movie_ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

movie_ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,107,5,NaN,Jurassic Park (1993)
1,6039,160,5,NaN,Forrest Gump (1994)
2,6039,602,5,NaN,2001: A Space Odyssey (1968)
3,6039,479,5,NaN,Contact (1997)


### CSR matrix를 직접 만들기

In [17]:
from scipy.sparse import csr_matrix

num_user = movie_ratings['user_id'].nunique()
num_movie = movie_ratings['movie_id'].nunique()

csr_data = csr_matrix((movie_ratings['counts'], (movie_ratings.user_id, movie_ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [26]:
from implicit.als import AlternatingLeastSquares

# os.environ['OPENBLAS_NUM_THREADS']='1'
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
# os.environ['MKL_NUM_THREADS']='1'

In [27]:
# gpu 사용
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=True, iterations=15, dtype=np.float32)

In [28]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [34]:
lee, gladiator = user_to_idx['lee'], movie_to_idx['Gladiator (2000)']
lee_vector, gladiator_vector = als_model.user_factors[lee], als_model.item_factors[gladiator]

In [36]:
lee_vector = lee_vector.to_numpy()[0]
lee_vector

array([-0.9097053 ,  0.5835681 , -0.03250636, -0.32518846, -0.49257737,
        0.22280958,  0.53276634, -0.4735124 ,  0.7180543 , -1.0170677 ,
       -0.9248973 ,  0.5074393 ,  0.12099835,  0.46193388, -1.0449877 ,
        1.461816  ,  0.35455567, -1.2205908 ,  0.20949394, -0.49469364,
       -0.07143267,  0.9735603 ,  0.2754695 ,  0.04576629,  0.19204897,
        0.44494   , -0.12631573,  0.27952513, -0.5754534 , -0.3068358 ,
       -1.0821128 ,  0.1184556 , -0.35694513, -0.53721786, -1.2705562 ,
       -0.1375828 , -0.37634668,  1.0978074 ,  0.17155813,  0.50188094,
       -0.46312907, -0.28794086, -0.41724956, -0.47711965,  0.09186118,
        0.02502223, -0.04957604, -0.6152722 , -0.01446827,  0.05821874,
        0.21005332,  0.47294927,  0.6183899 , -0.65294594, -0.3956449 ,
        0.21686527, -0.36464354, -0.21918736, -0.95220274,  0.1672587 ,
       -0.9188308 ,  1.0914732 ,  0.3527259 , -0.50355184, -1.5394709 ,
        0.06197782,  0.9713907 , -0.22108097,  0.10606167, -0.10

In [37]:
gladiator_vector = gladiator_vector.to_numpy()[0]
gladiator_vector

array([-0.03654852,  0.0332708 ,  0.00590159, -0.03378538, -0.03493052,
        0.00163113,  0.02303922, -0.00977044,  0.00887368, -0.0492722 ,
        0.00139536,  0.04548629, -0.00509128,  0.01226065, -0.00862895,
       -0.00127468,  0.01444151, -0.04829864,  0.01877478, -0.03079595,
        0.00814949,  0.00044315, -0.01537673, -0.03269892, -0.01889881,
        0.02665805, -0.00257206,  0.00017146, -0.02720944,  0.00685574,
        0.01738754, -0.00512715, -0.02354539, -0.03170877, -0.02524367,
       -0.00297447,  0.00899101, -0.001828  ,  0.01487222,  0.01268658,
       -0.00226854,  0.01501793,  0.00633283, -0.02195106,  0.019882  ,
        0.00951867,  0.00922609, -0.01872333, -0.00678725, -0.02578736,
       -0.00350631, -0.02308637,  0.00197946, -0.00769805, -0.03255593,
        0.01659223, -0.00365537,  0.00104014, -0.02708898,  0.0017374 ,
       -0.010497  , -0.01209054, -0.00908929, -0.011359  , -0.01178147,
       -0.01444526,  0.03232795,  0.03217955, -0.00681384, -0.01

In [38]:
np.dot(lee_vector, gladiator_vector)

0.57878685

In [39]:
exhale = movie_to_idx['Waiting to Exhale (1995)']
exhale_vector = als_model.item_factors[exhale]
exhale_vector = exhale_vector.to_numpy()[0]
np.dot(lee_vector, exhale_vector)

-0.018470453

### 내가 좋아하는 영화와 비슷한 영화를 추천

In [40]:
favorite_movie = '2001: A Space Odyssey (1968)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie

[(602, 1.0),
 (550, 0.65332896),
 (91, 0.6426645),
 (904, 0.5547285),
 (876, 0.4553493),
 (193, 0.4505332),
 (1097, 0.44572565),
 (680, 0.44488633),
 (74, 0.3691034),
 (651, 0.36171275)]

In [41]:
# artist_to_idx 를 뒤집어 index로 부터 artist 이름을 얻는 dict 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['2001: A Space Odyssey (1968)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Close Encounters of the Third Kind (1977)',
 'Clockwork Orange, A (1971)',
 '2010 (1984)',
 'Alien (1979)',
 'Brazil (1985)',
 'Blade Runner (1982)',
 'Day the Earth Stood Still, The (1951)',
 'Aliens (1986)']

In [42]:
# 비슷한 영화 이름 검색 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [43]:
get_similar_movie('Jumanji (1995)')

['Jumanji (1995)',
 'Hook (1991)',
 'Indian in the Cupboard, The (1995)',
 'Dragonheart (1996)',
 'Santa Clause, The (1994)',
 'Flubber (1997)',
 'NeverEnding Story II: The Next Chapter, The (1990)',
 'Small Soldiers (1998)',
 'Space Jam (1996)',
 'Borrowers, The (1997)']

### 내가 가장 좋아할 만한 영화들을 추천


In [44]:
user = user_to_idx['lee']
# recommend에서는 user*item CSR Matrix를 받음
movie_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True)
movie_recommended

[(346, 0.47224346),
 (175, 0.35572985),
 (22, 0.34958193),
 (87, 0.32287192),
 (124, 0.31964213),
 (92, 0.29672557),
 (150, 0.28675753),
 (372, 0.28147468),
 (904, 0.2788471),
 (317, 0.27280265)]

In [46]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Patriot, The (2000)',
 'Men in Black (1997)',
 'Back to the Future (1985)',
 'Braveheart (1995)',
 'Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Independence Day (ID4) (1996)',
 'X-Men (2000)',
 'Clockwork Orange, A (1971)',
 'Twelve Monkeys (1995)']

![nn](rubric_2.png)

# 회고
진행하는데 있어서 조금 갈피를 못잡았다. 코드를 하나씩 쳐가면서 했는데도 코드들이 익지 않았다. 그래도 계속 보다보면 괜찮아 질거라고 생각한다. gpu 를 사용하니 확실히 작업이 빨라져서 좋았다.
gpu 사용하면 2차원 매트릭스로 나와서 좀 헤맸지만 1차원 array로 변경하는 방법을 통해 문제를 해결 할 수 있었다. 데이터 타입에 대해서 공부를 좀 더 해야겠다. 마지막에 내가 가장 좋아할 만한 영화에서 예전에 보려고 했던 시계 태엽 오렌지 가 나와서 신기했다. 아직 감상은 못했지만 나에게 맞는편인가보다.
따라 쓰면서 데이터 전처리를 제대로 익혀야겠다.ㅣ